In [335]:
"""
Скрипт для обработки данных  с портала трудвсем для БЦК
"""

'\nСкрипт для обработки данных  с портала трудвсем для БЦК\n'

In [336]:
import pandas as pd
import numpy as np
import openpyxl
import json
import ast
import re
import os
import datetime
import time
from dateutil import parser
pd.options.mode.chained_assignment = None


In [337]:
def extract_data_from_list_cell(cell:str,lst_need_keys:list):
    """
    Функция для извлечения данных из словаря в ячейке датафрейма
    """
    try:
        
        lst_lang = ast.literal_eval(cell) # превращаем в список
        if lst_lang:            
            out_str_lst = [] # создаем список содержащий выходные строки
            for lang_dict in lst_lang:
                lst_lang_str = [] # список для хранения значений извлеченных из словаря
                for idx,key in enumerate(lst_need_keys):
                    lst_lang_str.append(lang_dict.get(key,None))             
                    
                lst_lang_str = [value for value in lst_lang_str if value] # отбрасываем None
                single_lang_str = ','.join(lst_lang_str) # создаем строку для одного языка
                out_str_lst.append(single_lang_str)
            return ';'.join(out_str_lst)
                               
        else:
            return 'Не указано'
        

    except FileNotFoundError:
        return 'Не удалось обработать содержимое ячейки'

    

In [338]:
def clear_tag(cell):
    """
    Функция для очистки текста от тегов HTML
    """
    value = str(cell)
    if value != 'nan':        
        result = re.sub(r'<.*?>','',value)
        result = re.sub(r'&[a-z]*?;','',result)
        return result
    else:
        return None

def clear_bonus_tag_br(cell):
    """
    Функция для очистки данных в колонке Дополнительные бонусы
    """
    cell = str(cell)

    if cell !='nan':
        value = str(cell)
        result = re.sub(r'<.*?>','.',value)

        return result
    else:
        return None

In [339]:
def convert_date(cell):
    """
    Функция конвертации строки содержащей дату и время
    """
    value = str(cell)
    try:        
        if value !='nan':
            date_time = parser.parse(value).date() # извлекаем дату
            date_time = datetime.datetime.strftime(date_time,'%d.%m.%Y') # конфертируем в нужный формат
            return date_time

        else:
            return None
    except:
        return 'Не удалось обработать содержимое ячейки'
    

In [340]:
def prepare_data_vacancy(df:pd.DataFrame,dct_name_columns:dict,lst_columns:list)->pd.DataFrame:
    """
    Функция для обработки датафрейма с данными работы в России
    """
    df = df[dct_name_columns.keys()]
    df.rename(columns=dct_name_columns,inplace=True)
    # Обрабатываем обычные колонки
    
    df['Дополнительные бонусы'] = df['Дополнительные бонусы'].apply(clear_bonus_tag_br)
    df['Требования'] = df['Требования'].apply(clear_tag)
    df['Обязанности'] = df['Обязанности'].apply(clear_tag)
    
    # Числовые
    lst_number_columns = ['Требуемый опыт работы в годах','Минимальная зарплата','Максимальная зарплата','Количество рабочих мест']
    df[lst_number_columns] = df[lst_number_columns].fillna(0)
    df[lst_number_columns] = df[lst_number_columns].astype(int,errors='ignore')
  
    # Временные
    
    df['Дата размещения вакансии'] = df['Дата размещения вакансии'].apply(convert_date)
    df['Дата изменения вакансии'] = df['Дата изменения вакансии'].apply(convert_date)
    
    # Категориальные
    df['Квотируемое место'] = df['Квотируемое место'].apply(lambda x: 'Квотируемое место' if x == 'true' else None)
    df['Требуется медкнижка'] = df['Требуется медкнижка'].apply(lambda x:'Требуется медкнижка' if x == 'true' else None)
    df['Статус проверки вакансии'] = df['Статус проверки вакансии'].replace(dct_status_vacancy)
    # Начинаем извлекать данные из сложных колонок с json
    # данные по работодателю
    df['КПП работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('kpp','Не указано'))
    df['ОГРН работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('ogrn','Не указано'))
    df['Контактный телефон'] = df['Данные компании'].apply(lambda x: json.loads(x).get('phone','Не указано'))
    df['Email работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('email','Не указано'))
    df['Профиль работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('url','Не указано'))
    
    # Обрабатываем колонку с языками
    df['Требуемые языки'] = df['Данные по языкам'].apply(lambda x:extract_data_from_list_cell(x,['code_language','level']))
    df['Требуемые хардскиллы'] = df['Данные по хардскиллам'].apply(lambda x:extract_data_from_list_cell(x,['hard_skill_name']))
    df['Требуемые софтскиллы'] = df['Данные по софтскиллам'].apply(lambda x:extract_data_from_list_cell(x,['soft_skill_name']))
    
    df.drop(columns=['Данные компании','Данные по языкам','Данные по хардскиллам','Данные по софтскиллам'],inplace=True)
    
    df = df.reindex(columns=lst_columns)
    
    return df
    
    

In [341]:
region = 'Приморский край'
file_data = 'data/vacancy.csv'
file_org = 'data/company.xlsx'
end_folder = 'data'

In [342]:
# колонки которые нужно оставить и переименовать
dct_name_columns = {'busy_type':'Тип занятости','contact_person':'Контактное лицо','date_create':'Дата размещения вакансии',
                    'date_modify':'Дата изменения вакансии','education':'Образование','education_speciality':'Требуемая специализация','is_quoted':'Квотируемое место',
                    'need_medcard':'Требуется медкнижка',
                   'other_vacancy_benefit':'Дополнительные бонусы','position_requirements':'Требования',
                    'position_responsibilities':'Обязанности','regionName':'Регион','required_experience':'Требуемый опыт работы в годах',
                    'retraining_capability':'Возможность переподготовки','required_certificates':'Требуемые доп. документы',
                   'required_drive_license':'Требуемые водительские права','retraining_grant':'Стипендия',
                    'retraining_grant_value':'Размер стипендии','salary':'Зарплата','salary_min':'Минимальная зарплата','salary_max':'Максимальная зарплата',
                    'schedule_type':'График работы','social_protected_ids':'Социально защищенная категория',
                   'source_type':'Источник вакансии','status':'Статус проверки вакансии','transport_compensation':'Компенсация транспорт',
                    'vacancy_address_additional_info':'Доп информация по адресу вакансии','vacancy_address':'Адрес вакансии',
                    'vacancy_address_latitude':'Долгота адрес вакансии','vacancy_address_longitude':'Широта адрес вакансии',
                   'vacancy_benefit_ids':'Бонусы','vacancy_name':'Вакансия','work_places':'Количество рабочих мест','professionalSphereName':'Сфера деятельности',
                    'full_company_name':'Полное название работодателя','company_inn':'ИНН работодателя','company':'Данные компании',
                    'languageKnowledge':'Данные по языкам','hardSkills':'Данные по хардскиллам','softSkills':'Данные по софтскиллам'}

In [343]:
dct_status_vacancy = {'ACCEPTED':'Данные вакансии проверены работодателем','AUTOMODERATION':'Автомодерация',
                      'REJECTED':'Отклонено','CHANGED':'Статус вакансии изменен',
                      'WAITING':'Ожидает подтверждения',}

In [344]:
t = time.localtime()  # получаем текущее время
current_time = time.strftime('%H_%M_%S', t)
current_date = time.strftime('%d_%m_%Y',t)

In [345]:
df = pd.read_csv(file_data,encoding='UTF-8',sep='|',dtype=str)


In [346]:
df.shape

(40330, 78)

In [347]:
company_df = pd.read_excel(file_org,dtype=str)

In [348]:
company_df

,Краткое название организации,ИНН
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АРСЕНЬЕВСКАЯ АВИАЦИОННАЯ...",2501002394
1,АКЦИОНЕРНОЕ ОБЩЕСТВО ВЛАДИВОСТОКСКОЕ ПРЕДПРИЯТ...,2504000733
2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВАРЯГ-ТЕХСЕРВИС""",2539032214
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЦЕНТР СУДОРЕМОНТА ""ДАЛЬЗ...",2536210349
4,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МАРТЕН""",2536013118
5,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""30 СУДОРЕМОНТНЫЙ ЗАВОД""",2512304968
6,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СЛАВЯНСКИЙ СУД...",2531001535
7,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ДАЛЬПРИБОР""",2539008807
8,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ИЗУМРУД""",2539028264
9,"ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""741 РЕМОНТНЫЙ З...",2536217947


In [349]:
lst_columns = ['Дата размещения вакансии','Дата изменения вакансии','Регион','Вакансия','Сфера деятельности','Количество рабочих мест',
               'Зарплата','Минимальная зарплата','Максимальная зарплата','График работы','Тип занятости','Образование','Требуемая специализация',
               'Требования','Обязанности','Бонусы','Дополнительные бонусы','Возможность переподготовки','Стипендия','Размер стипендии','Компенсация транспорт',
               'Квотируемое место','Социально защищенная категория',
               'Требуемый опыт работы в годах','Требуется медкнижка','Требуемые доп. документы','Требуемые водительские права',
               'Требуемые языки','Требуемые хардскиллы','Требуемые софтскиллы',
               'Источник вакансии','Статус проверки вакансии','Полное название работодателя','Адрес вакансии','Доп информация по адресу вакансии',
               'ИНН работодателя','КПП работодателя','ОГРН работодателя','Контактное лицо','Контактный телефон','Email работодателя',
               'Профиль работодателя','Долгота адрес вакансии','Широта адрес вакансии']

In [350]:
df = df[df['regionName'] == region]

In [351]:
df.shape

(13074, 78)

In [352]:
all_status_prepared_df = prepare_data_vacancy(df,dct_name_columns,lst_columns) # получаем обработанный датафрейм со всеми статусами вакансий



In [353]:
all_status_prepared_df.shape

(13074, 44)

In [354]:
prepared_df = all_status_prepared_df[all_status_prepared_df['Статус проверки вакансии'] == 'Данные вакансии проверены работодателем']
union_company_df_columns = list(prepared_df.columns).insert(0,'Организация')
union_company_df = pd.DataFrame(columns=union_company_df_columns)

In [355]:
prepared_df.shape

(13056, 44)

In [356]:
dct_df = dict() # словарь для хранения данных для каждого инн
used_name_sheet = set() # множество для хранения использованных названий листов

In [357]:
org_folder = f'{end_folder}/Вакансии по организациям/{current_date}' # создаем папку куда будем складывать вакансии по организациям
if not os.path.exists(org_folder):
    os.makedirs(org_folder)


# Собираем датафреймы по ИНН
for idx,row in enumerate(company_df.itertuples()):
    name_company = row[1] # название компании
    
    inn_company = row[2] # инн компании
    temp_df = prepared_df[prepared_df['ИНН работодателя'] ==inn_company] # фильтруем по инн
    temp_df.sort_values(by=['Вакансия'],inplace=True) # сортируем
    name_company = re.sub(r'[\r\b\n\t<>:"?*|\\/]', '_', name_company)# очищаем от лишних символов

    temp_df.to_excel(f'{org_folder}/{name_company}.xlsx',index=False) # сохраняем
    # создаем отдельный файл в котором будут все вакансии по выбранным компаниям
    temp_df.insert(0,'Организация',name_company)
    union_company_df = pd.concat([union_company_df,temp_df],ignore_index=True)
    
    
    

In [358]:
union_company_df

,Организация,Дата размещения вакансии,Дата изменения вакансии,Регион,Вакансия,Сфера деятельности,Количество рабочих мест,Зарплата,Минимальная зарплата,Максимальная зарплата,...,Доп информация по адресу вакансии,ИНН работодателя,КПП работодателя,ОГРН работодателя,Контактное лицо,Контактный телефон,Email работодателя,Профиль работодателя,Долгота адрес вакансии,Широта адрес вакансии
0,АКЦИОНЕРНОЕ ОБЩЕСТВО _АРСЕНЬЕВСКАЯ АВИАЦИОННАЯ...,31.01.2024,28.02.2024,Приморский край,Авиационный механик(техник) по приборам и элек...,Машиностроение,10,от 50000,50000,0,...,NaN,2501002394,250101001,1022500510350,Анна Викторовна Кононова,+7(42361)45047,e.maksimenko@aacprogress.ru,https://trudvsem.ru/company/1022500510350,44.153185,133.26017
1,АКЦИОНЕРНОЕ ОБЩЕСТВО _АРСЕНЬЕВСКАЯ АВИАЦИОННАЯ...,10.11.2023,28.02.2024,Приморский край,Авиационный техник,Машиностроение,1,от 50000,50000,0,...,NaN,2501002394,250101001,1022500510350,Анна Викторовна Кононова,+7(42361)45047,e.maksimenko@aacprogress.ru,https://trudvsem.ru/company/1022500510350,44.153185,133.26017
2,АКЦИОНЕРНОЕ ОБЩЕСТВО _АРСЕНЬЕВСКАЯ АВИАЦИОННАЯ...,20.10.2023,06.03.2024,Приморский край,Администратор вычислительной сети - 1,Машиностроение,1,от 30000,30000,0,...,NaN,2501002394,250101001,1022500510350,Кононова Анна Викторовна,+7(42361)45047,e.maksimenko@aacprogress.ru,https://trudvsem.ru/company/1022500510350,44.153185,133.26017
3,АКЦИОНЕРНОЕ ОБЩЕСТВО _АРСЕНЬЕВСКАЯ АВИАЦИОННАЯ...,26.02.2024,26.02.2024,Приморский край,Администратор вычислительной сети 3 категории ...,Машиностроение,1,от 40000,40000,0,...,NaN,2501002394,250101001,1022500510350,Кононова Анна Викторовна,+7(42361)45047,e.maksimenko@aacprogress.ru,https://trudvsem.ru/company/1022500510350,44.153185,133.26017
4,АКЦИОНЕРНОЕ ОБЩЕСТВО _АРСЕНЬЕВСКАЯ АВИАЦИОННАЯ...,30.01.2024,26.02.2024,Приморский край,Аппаратчик полимеризации 3-5 разряд 15 30.01.2024,Машиностроение,15,от 50000,50000,0,...,NaN,2501002394,250101001,1022500510350,Анна Викторовна Кононова,+7(42361)45047,e.maksimenko@aacprogress.ru,https://trudvsem.ru/company/1022500510350,44.153185,133.26017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,АКЦИОНЕРНОЕ ОБЩЕСТВО _НАХОДКИНСКИЙ СУДОРЕМОНТН...,31.01.2023,12.03.2024,Приморский край,Специалист транспортной безопасности,Юриспруденция,8,от 40000,40000,50000,...,"692913, г Находка, пр-кт Находкинский, д. 59",2508001431,250801001,1022500704456,Анастасия Владиславовна,+7(914)7090708,nsry@nsry.ru,https://trudvsem.ru/company/1022500704456,42.80149,132.87619
457,АКЦИОНЕРНОЕ ОБЩЕСТВО _НАХОДКИНСКИЙ СУДОРЕМОНТН...,01.01.2023,12.03.2024,Приморский край,Старший электромеханик (судовой),"Транспорт, автобизнес, логистика, склад, ВЭД",1,от 45000,45000,50000,...,"692913, г Находка, пр-кт Находкинский, д. 59",2508001431,250801001,1022500704456,Анастасия Владиславовна,+7(914)7090708,nsry@nsry.ru,https://trudvsem.ru/company/1022500704456,42.80149,132.87619
458,АКЦИОНЕРНОЕ ОБЩЕСТВО _НАХОДКИНСКИЙ СУДОРЕМОНТН...,13.02.2024,13.02.2024,Приморский край,Токарь,Производство,1,от 60000,60000,0,...,NaN,2508001431,250801001,1022500704456,Анастасия Владиславовна,+7(914)7090708,nsry@nsry.ru,https://trudvsem.ru/company/1022500704456,42.80149,132.87619
459,АКЦИОНЕРНОЕ ОБЩЕСТВО _НАХОДКИНСКИЙ СУДОРЕМОНТН...,21.09.2023,12.03.2024,Приморский край,Электромонтер по ремонту и обслуживанию электр...,Электроэнергетика,4,от 40000,40000,60000,...,NaN,2508001431,250801001,1022500704456,Анастасия Владиславовна,+7(914)7090708,nsry@nsry.ru,https://trudvsem.ru/company/1022500704456,42.80149,132.87619


In [359]:
union_company_df.to_excel(f'{org_folder}/Общий файл.xlsx',index=False)

In [360]:
with pd.ExcelWriter(f'{end_folder}/Данные по региону от {current_time}.xlsx') as writer:
    prepared_df.to_excel(writer,sheet_name='Только подтвержденные вакансии',index=0)
    df.to_excel(writer,sheet_name='Необработанные данные',index=1)
    all_status_prepared_df.to_excel(writer,sheet_name='Вакансии со всеми статусами',index=2)

In [361]:
# Свод по региону 


In [376]:
# Свод по количеству рабочих мест по отраслям

svod_vac_reg_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]})
svod_vac_reg_region_df = svod_vac_reg_region_df.droplevel(level=0,axis=1) # убираем мультииндекс
svod_vac_reg_region_df.sort_values(by=['sum'],ascending=False,inplace=True)
svod_vac_reg_region_df.loc['Итого'] = svod_vac_reg_region_df['sum'].sum()
svod_vac_reg_region_df.rename(columns={'sum':'Количество вакансий'},inplace=True)
svod_vac_reg_region_df=svod_vac_reg_region_df.reset_index()

In [377]:
# Свод по количеству рабочих мест по организациям
svod_vac_org_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]})
svod_vac_org_region_df = svod_vac_org_region_df.droplevel(level=0,axis=1) # убираем мультииндекс
svod_vac_org_region_df.sort_values(by=['sum'],ascending=False,inplace=True)
svod_vac_org_region_df.loc['Итого'] = svod_vac_org_region_df['sum'].sum()
svod_vac_org_region_df.rename(columns={'sum':'Количество вакансий'},inplace=True)
svod_vac_org_region_df=svod_vac_org_region_df.reset_index()

In [388]:
# Свод по средней и медианной минимальной зарплате для сфер деятельности
svod_shpere_pay_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности'],
              values = ['Минимальная зарплата'],
              aggfunc={'Минимальная зарплата':[np.mean,np.median]},
                                        fill_value=0)
svod_shpere_pay_region_df = svod_shpere_pay_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_pay_region_df = svod_shpere_pay_region_df.astype(int,errors='ignore')
svod_shpere_pay_region_df.columns = ['Средняя ариф. минимальная зп','Медианная минимальная зп']
svod_shpere_pay_region_df = svod_shpere_pay_region_df.reset_index()




In [392]:
# Свод по средней и медианной минимальной зарплате для работодателей
svod_org_pay_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Сфера деятельности'],
              values = ['Минимальная зарплата'],
              aggfunc={'Минимальная зарплата':[np.mean,np.median]},
                                        fill_value=0)
svod_org_pay_region_df = svod_org_pay_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_pay_region_df = svod_org_pay_region_df.astype(int,errors='ignore')
svod_org_pay_region_df.columns = ['Средняя ариф. минимальная зп','Медианная минимальная зп']
svod_org_pay_region_df = svod_org_pay_region_df.reset_index()


In [396]:
# Свод по требуемому образованию для сфер деятельности
svod_shpere_educ_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','Образование'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_educ_region_df = svod_shpere_educ_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_educ_region_df = svod_shpere_educ_region_df.astype(int,errors='ignore')
svod_shpere_educ_region_df.columns = ['Количество вакансий']
svod_shpere_educ_region_df = svod_shpere_educ_region_df.reset_index()

In [398]:
# Свод по требуемому образованию для работодателей
svod_org_educ_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Образование'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_educ_region_df = svod_org_educ_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_educ_region_df = svod_org_educ_region_df.astype(int,errors='ignore')
svod_org_educ_region_df.columns = ['Количество вакансий']
svod_org_educ_region_df = svod_org_educ_region_df.reset_index()

In [400]:
# Свод по графику работы для сфер деятельности
svod_shpere_schedule_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','График работы'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_schedule_region_df = svod_shpere_schedule_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_schedule_region_df = svod_shpere_schedule_region_df.astype(int,errors='ignore')
svod_shpere_schedule_region_df.columns = ['Количество вакансий']
svod_shpere_schedule_region_df = svod_shpere_schedule_region_df.reset_index()

In [402]:
# Свод по графику работы для работодателей
svod_org_schedule_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','График работы'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_schedule_region_df = svod_org_schedule_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_schedule_region_df = svod_org_schedule_region_df.astype(int,errors='ignore')
svod_org_schedule_region_df.columns = ['Количество вакансий']
svod_org_schedule_region_df = svod_org_schedule_region_df.reset_index()

In [404]:
# Свод по типу занятости для сфер деятельности
svod_shpere_type_job_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','Тип занятости'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_type_job_region_df = svod_shpere_type_job_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_type_job_region_df = svod_shpere_type_job_region_df.astype(int,errors='ignore')
svod_shpere_type_job_region_df.columns = ['Количество вакансий']
svod_shpere_type_job_region_df = svod_shpere_type_job_region_df.reset_index()

In [413]:
# Свод по типу занятости для работодателей
svod_org_type_job_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Тип занятости'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_type_job_region_df = svod_org_type_job_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_type_job_region_df = svod_org_type_job_region_df.astype(int,errors='ignore')
svod_org_type_job_region_df.columns = ['Количество вакансий']
svod_org_type_job_region_df = svod_org_type_job_region_df.reset_index()

#svod_org_type_job_region_df.sort_values(by=['Полное название работодателя','Тип занятости','Количество вакансий'],
 #                                      ascending=[False,False,False])

C:\Users\1\AppData\Local\Temp\ipykernel_17912\141711334.py:12: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only.
  svod_org_type_job_region_df.apply(lambda x:x.sort_values('Количество вакансий'))


ValueError: No axis named Количество вакансий for object type Series

In [407]:
svod_org_type_job_region_df

,Полное название работодателя,Тип занятости,Количество вакансий
0,"""АЗИАТСКО-ТИХООКЕАНСКИЙ БАНК"" (АКЦИОНЕРНОЕ ОБЩ...",Полная занятость,2
1,"""ДАЛЬНЕВОСТОЧНЫЙ""",Полная занятость,40
2,15 ЦЕНТР СВЯЗИ СПЕЦИАЛЬНОГО НАЗНАЧЕНИЯ ФЕДЕРАЛ...,Полная занятость,3
3,Bayer,Полная занятость,1
4,Biglion,Полная занятость,1
...,...,...,...
2695,федеральное государственное бюджетное образова...,Сезонная,1
2696,федеральное казенное профессиональное образова...,Полная занятость,1
2697,"филиал ""Восточный""",Полная занятость,1
2698,"филиал ""Партизанский"" акционерного общества ""П...",Полная занятость,13


In [408]:
# Свод по квотируемым местам для сфер деятельности
svod_shpere_quote_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','Квотируемое место'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_quote_region_df = svod_shpere_quote_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_quote_region_df = svod_shpere_quote_region_df.astype(int,errors='ignore')
svod_shpere_quote_region_df.columns = ['Количество вакансий']
svod_shpere_quote_region_df = svod_shpere_quote_region_df.reset_index()

In [410]:
# Свод по квотируемым местам для работодателей
svod_org_quote_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Квотируемое место'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_quote_region_df = svod_org_quote_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_quote_region_df = svod_org_quote_region_df.astype(int,errors='ignore')
svod_org_quote_region_df.columns = ['Количество вакансий']
svod_org_quote_region_df = svod_org_quote_region_df.reset_index()

In [411]:
svod_org_quote_region_df

,Полное название работодателя,Квотируемое место,Количество вакансий
0,"""АЗИАТСКО-ТИХООКЕАНСКИЙ БАНК"" (АКЦИОНЕРНОЕ ОБЩ...",Квотируемое место,2
1,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ПРИМОРС...",Квотируемое место,1
2,АДМИНИСТРАЦИЯ ЧЕРНИГОВСКОГО МУНИЦИПАЛЬНОГО ОКР...,Квотируемое место,1
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""322 АВИАЦИОННЫЙ РЕМОНТНЫ...",Квотируемое место,1
4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""44 РЕМОНТНЫЙ ЗАВОД РАДИО...",Квотируемое место,1
...,...,...,...
474,муниципальное бюджетное общеобразовательное уч...,Квотируемое место,1
475,муниципальное бюджетное общеобразовательное уч...,Квотируемое место,1
476,муниципальное бюджетное общеобразовательное уч...,Квотируемое место,1
477,муниципальное общеобразовательное бюджетное уч...,Квотируемое место,1


In [365]:
print('Lindy Booth!')

Lindy Booth!
